In [2]:
import numpy as np
import json
from typing import List
import os, os.path
from collections import defaultdict

Got a dataset from Slay The Spire official discord where Baalor posted data about around 700 of his runs in 2022.

### Can we deterministically say what's the state of the deck at any floor?


In [4]:
data = json.load(open("C:/Users/lgerretsen/perso/perso/sts/2019-05-31-00-53#1028.json", "r"))


In [14]:
sample = data[0]
assert set(sample.keys()) == {'event'}
sample = sample["event"]
sample.keys()

dict_keys(['gold_per_floor', 'floor_reached', 'playtime', 'items_purged', 'score', 'play_id', 'local_time', 'is_ascension_mode', 'campfire_choices', 'neow_cost', 'seed_source_timestamp', 'circlet_count', 'master_deck', 'relics', 'potions_floor_usage', 'damage_taken', 'seed_played', 'potions_obtained', 'is_trial', 'path_per_floor', 'character_chosen', 'items_purchased', 'campfire_rested', 'item_purchase_floors', 'current_hp_per_floor', 'gold', 'neow_bonus', 'is_prod', 'is_daily', 'chose_seed', 'campfire_upgraded', 'win_rate', 'timestamp', 'path_taken', 'build_version', 'purchased_purges', 'victory', 'max_hp_per_floor', 'card_choices', 'player_experience', 'relics_obtained', 'event_choices', 'is_beta', 'boss_relics', 'items_purged_floors', 'is_endless', 'potions_floor_spawned', 'killed_by', 'ascension_level'])

In [15]:
sample["neow_bonus"]

'REMOVE_TWO'

In [16]:
sample["items_purged_floors"]


[7]

In [24]:
idx = 0
while 1:
    sample = data[idx]["event"]
    if sample["victory"]:
        break
    idx += 1
json.dump(sample, open("./example_vanilla.run", "w"), indent=4)